In [ ]:
# https://www.analyticsvidhya.com/blog/2018/09/deep-learning-video-classification-python/#respond
import cv2 # for capturing videos 
import math # for the mathematical operations required by our program 
import matplotlib.pyplot as plt # helps us plot the images
import pandas as pd 
%matplotlib inline
from keras.preprocessing import image # does the preprocessing for the images
import numpy as np # for more mathematical operations 
from keras.utils import np_utils
from skimage.transform import resize # Does the resizing for the images 

# First we have to load the video and convert it into frames. 
count = 0
videoFile = "https://www.youtube.com/shorts/NjwktXZOCqs"
cap = cv2.VideoCapture(videoFile) # gets the video from the given path 
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

# Process prints done once the frames have been created. 
    
count = 0
videoFile = ""
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened(https://www.youtube.com/shorts/NjwktXZOCqs)):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

data = pd.read_csv('Classification.csv') # This reads the csv file 
data.head() # This prints the first five rows of the file 
# The mapping file contains the name of each image, or Image_ID, as well as the corresponding class, which is referenced in the model_class.json file. 
X = [] # Creates an empty array 
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img) # stores each image in array X
X = np.array(X) # converts the list to an array 

# Now we need two things to train our model, training images, and their corresponding class 

test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)


from keras.utils import np_utils
train_y = np_utils.to_categorical(data.Class)
test_y = np_utils.to_categorical(test.Class)

# Using an input image of a shape that is 224 by 224 by 3 so are images need to be reshaped. 
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

# Preprocessing the image to allow the model to perform better. 
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')
test_image = preprocess_input(test_image, mode='tf')
# Checking a validation set to check the performance of the model on unseen images. 
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)

# using the VGG16 pretrained model for building the model. 
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

# Loading the VGG16 pretrained model and storing it as the base model 
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# We will make predictions using this model for x_train and x_valid, get the features, and then use the features to retrain the model. 
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
test_image = base_model.predict(test_image)
# Reshaping X_train and X_valid to pass it through out nueral network. 
X_train = X_train.reshape(208, 7*7*512)
X_valid = X_valid.reshape(90, 7*7*512)
test_image = test_image.reshape(186, 7*7*512)

# Preprocess the images and make them zero-centered to help the model to converge faster. 
train = X_train/X_train.max()
X_valid = X_valid/X_train.max()
test_image = test_image/test_image.max()

# Building the model. 
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(3, activation='softmax'))            # output layer

# Compiling the modle 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(data.Class), data.Class)  # computing weights of different classes

from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss

#Training the model 
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid), class_weight=class_weights, callbacks=callbacks_list)

In [ ]:
model.load_weights("weights.best.hdf5")
model.compile(loss-'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

scores=model.evaluate(test_image, test_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))